## What is ARR?

**ARR** stands for **Annual Recurring Revenue**, which is [revenue](https://www.investopedia.com/terms/r/revenue.asp) viewed on an annual basis that is expected to renew. To get **ARR**, start with the **Total Contract Value** and annualize it to get **Annual Contract Value**, then take the **ACV** and filter out what is deemed non-recurring to get **ARR**. Start with this example contract of two years totalling $20k$.

$\text{Contract 1}$
 
|Item|Amount|Start Date|End Date|
|---|---|---|---|
|Software Feature 1|$4000|01/01/2022|12/31/2023|
|Software Feature 2|$4000|01/01/2022|12/31/2023|
|Software Feature 3|$4000|01/01/2022|12/31/2023|
|Software Feature 4|$4000|01/01/2022|12/31/2023|
|Onboarding|$4000|01/01/2022|12/31/2023|

$\text{Contract 1}$ has 5 lines in it. 4 out of the 5 lines would be considered renewable. Once the contract is *up for renewal*, the customer can repurchase the 4 software features, but they would not repurchase the onboarding item, because this onboarding is only needed when the customer is new to the product. That puts the **ARR** at $8k$, once you annualize and deduct non-renewables.

$\begin{matrix}
 TCV & \to & ACV & \to & ARR  \\
 20k & \to & 10k & \to & 8k
\end{matrix}$

## Annualizing

The example above is simple. Take the two years at $20k$ and multiply $.5$ to get the annualized amount. However, the complexity comes with details. The basic formula to annualize is as follows.

$\text{ACV} = \text{TCV} * \frac{\text{Time Interval (Year)}}{\text{Contract Term Length}}$

The **Time Interval (Year)** is the number used to represent 1 year. Depending on what interval you are annualizing at, this can be represented a few different ways.

$
\begin{matrix}
\text{Year} & \to & 1 \\
\text{Quarter} & \to & 4 \\
\text{Month} & \to & 12 \\
\text{Day} & \to & 365
\end{matrix}
$

Simplifying the fraction for the $20k$ 2 year example, you still end up multiplying the $20k$ by $\frac{1}{2}$. More creative contracts can lead to unexpected results depending on which interval is chosen.

$
\begin{matrix}
\text{Year} & \to & \text{20,000} * \frac{1}{2} \\\\
\text{Quarter} & \to & \text{20,000} * \frac{4}{8} \\\\
\text{Month} & \to & \text{20,000} * \frac{12}{24} \\\\
\text{Day} & \to & \text{20,000} * \frac{365}{730}
\end{matrix}
$

<div class="alert alert-block alert-info">
Take these <em>what if</em> scenarios and consider what happens:

- What if annualizing by day on a leap year?
- What if the contract length is less than 1 year?
- What if the contract length is multiple years?
   - And that contract length is over a leap year?
- What if the contract ramps?
   - Meaning the customer is expected to pay an increased amount in the second year for the same product.
- What if the contract is cancelled?
   - Cancelled fully?
   - Cancelled Partially?
</div>

To consider these conundrums, let's jump into some python.

### The Code

The basis of our code will start with a contract. Contracts contain information that tell us about the agreement between the company and the customer. It will have information at the 'header' level, and it will have information at the 'line' level. Contract lines are a many to one relationship with the contract header. Combine all that together, and it makes a contract.

<div class="alert alert-block alert-info">
I've written functions and classes used across different notebooks in the `arr` folder so it can be imported in as a package.
</div>

#### Contracts

In [1]:
from arr import Contract, ContractHeader, ContractLine
from inspect import getsource
from IPython.display import Code

# Let's look at the Contract dataclasses
Code(getsource(ContractHeader), language="python3")

@dataclass
class ContractHeader:
    """Dataclass to handle the header of a contract.

    Args:
        amount (int): Total contract value.
        start_date (date): The start date of the Contract.
        end_date (date): The end date of the Contract.
        booking_date (date): The date the contract was booked.
            Defaults to `None`, if nothing is provided, but
            gets replaced to the `start_date` in the `__post_init__()`
    """

    amount: int
    start_date: date
    end_date: date
    booking_date: date = None

    def __post_init__(self):
        if self.booking_date is None:
            self.booking_date = self.start_date

Pretty basic setup, notice the `__post_init__()`. If a bookings date is not provided, the default value will be `None`. The `__post__init()` will set the bookings date to be the start date.

In [2]:
Code(getsource(ContractLine), language="python3")

@dataclass
class ContractLine:
    """Dataclass to handle contract lines.

    Args:
        amount (int): Total value of the line.
        start_date (date): The Start Date of the Contract.
        end_date (date): The End Date of the Contract.
        product (int): The product purchased by customer.
        renewable (bool): indicator whether or not product
            is renewable.
    """

    amount: int
    start_date: date
    end_date: date
    product: str
    renewable: bool

Similar to `ContractHeader`, but added `product` and `renewable`. It's possible for contract line start and end dates to have different dates than the contract header start and end dates. An example of that would be a multi-year contract.

In [3]:
source = getsource(Contract)
Code(source[:source.find('def __repr__')], language="python3")

@dataclass
class Contract:
    """Bringing it together."""

    id: int
    header: ContractHeader
    lines: List[ContractLine]
    customer: str = None

One `Contract` is made up of the header (`ContractHeader`) and multiple lines (`ContractLine`). These dataclasses are representative of transactional records in a source system (SalesForce, NetSuite, SAP, etc.).


<div class="alert alert-block alert-info">
There is a lot more in the <code>Contract</code> dataclass. See the arr module for more info.
</div>

Let's look at some examples.

In [4]:
from datetime import date

example_contract1 = Contract(
    1,
    ContractHeader(20_000, date(2024, 1, 1), date(2024, 12, 31)),
    [
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 1, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 2, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 3, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 4, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 5, False),
    ],
)

example_contract2 = Contract(
    2,
    ContractHeader(36_000, date(2024, 1, 1), date(2026, 12, 31)),
    [
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 1, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 2, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 3, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 4, True),
        ContractLine(4_000, date(2024, 1, 1), date(2024, 12, 31), 5, False),
        ContractLine(4_000, date(2025, 1, 1), date(2025, 12, 31), 1, True),
        ContractLine(4_000, date(2025, 1, 1), date(2025, 12, 31), 2, True),
        ContractLine(4_000, date(2025, 1, 1), date(2025, 12, 31), 3, True),
        ContractLine(4_000, date(2025, 1, 1), date(2025, 12, 31), 4, True),
    ],
)

print(example_contract1)
print("-" * 55)
print(example_contract2)

                      Example Customer                      
             Contract #1 - $20,000 - 2024-01-01             
                  2024-01-01 - 2024-12-31                   
************************************************************
product	start date	end date	amount	renewable
************************************************************
1	2024-01-01	2024-12-31	$4,000	True
2	2024-01-01	2024-12-31	$4,000	True
3	2024-01-01	2024-12-31	$4,000	True
4	2024-01-01	2024-12-31	$4,000	True
5	2024-01-01	2024-12-31	$4,000	False
************************************************************

-------------------------------------------------------
                      Example Customer                      
             Contract #2 - $36,000 - 2024-01-01             
                  2024-01-01 - 2026-12-31                   
************************************************************
product	start date	end date	amount	renewable
************************************************************
1

#### Annualize

Onto annualizing. The `annualize` function will take $\text{TCV}\to\text{ACV}$.

<div class="alert alert-block alert-info">
For this writeup, I've given the function a lot of flexibility to show the different ways and pitfalls that are possible. <b>In practice, the flexibility would be removed to keep boundaries and simplification in your annualization calc.</b> It's a lot to unpack. Take time to read it in the <a href="arr/annualize.py">arr/annualize.py</a> file, I'll break it down here.</div>

In [5]:
from arr import annualize, explain_code
# explain_code is a custom generator, which will
# display a function in blocks from top to bottom.

In [6]:
annualize_code = explain_code(annualize)

# The docstring.
next(annualize_code)

Takes the input of `ContractHeader or `ContractLine` dataclass and annualizes tcv.

    Args:
        contract (ContractHeader | Contract Line): See dataclasses.
        period (date): The date in which we are looking at the ARR.
            Example, what is the ACV on Jan 31, 2024. Jan 31, 2024
            would be the `period`.
        interval_str (INTERVAL): The options to chose from when deciding
            what interval to multiply `tcv` by. Possible options are:
            "Year", "Quarter", "Month", "Day".
        generalize_leap_year (bool): Toggle to decide who to handle leap years.
            `True` will not consider leap years and set all years to 365 days.
            `False` will call `calendar.isleap(year)` of the `period` value
            to determine if the year has 366 days in it.
        print_details (bool): Optional if you want to see details
            of some of the args and variables.

    Returns:
        int: Returns Annual Contract Value. The annualized amount of
            Total Contract Value.

Most of the docstring is self explainable. Except for the `interval_str` type hint, so I'll bring that in to show what this arg is expecting.

In [7]:
from arr.annualize import INTERVAL
INTERVAL

typing.Literal['Year', 'Quarter', 'Month', 'Day']

In [8]:
next(annualize_code)

# 1
    if generalize_leap_year:
        day = 365
    else:
        day = 365 + isleap(period.year)

1. The `generalize_leap_year` argument does exactly that. If set to `True`, all years are considered at 365 days. If set to `False`, leap years are considered. For example, 2024 will have 366 days, while 2025 will have 365 days. The `generalize_leap_year` will also be used to deduct leap years from the denominator if annualizing by days. See `#4` for that information. The `isleap` function is from the [calendar](https://docs.python.org/3/library/calendar.html#calendar.isleap) library. It returns a `True` or `False`, if it is a leap year or not. Because `bool` is a subclass of an `int`, it can be sum'd. `True == 1` and `False == 0`. [docs](https://docs.python.org/3/library/stdtypes.html#boolean-type-bool).

In [9]:
next(annualize_code)

# 2
    interval_mapping = {"Year": 1, "Quarter": 4, "Month": 12, "Day": day}
    time_interval = interval_mapping[interval_str]

2. After getting the `day` variable from the leap year check, it specifies what the numerator (**Time Interval (Year)**) will be based on the possible options in the `INTERVAL` variable.

$\text{TCV} * \frac{\textbf{Time Interval (Year)}}{\text{Contract Term Length}}$


In [10]:
next(annualize_code)

# 3
    contract_term = get_contract_term(
        contract.start_date, contract.end_date, generalize_leap_year, interval_str
    )

3. This step will determine the denominator.

$\text{TCV} * \frac{\text{Time Interval (Year)}}{\textbf{Contract Term Length}}$

I've broken up the logic for the denominator into it's own function, `get_contract_term`. We'll take a quick detour.

##### Get Contract Term Function

In [11]:
from arr import get_contract_term
get_contract_term_code = explain_code(get_contract_term)
next(get_contract_term_code)

Primary use is to be nested in the `annualize` func.

    Will output the denominator of the ARR formula (Contract Term).

    Args:
        start_date (date): Start date of contract.
        end_date (date): End date of contract.
        generalize_leap_year (bool): Hard code 365 or add the extra day if `True`.
        interval_str (INTERVAL): see `INTERVAL` variable for possible options.

    Returns:
        int: Contract Term #.

Same arguments we've seen before. Will return an int that is the bottom of our fraction.

In [12]:
next(get_contract_term_code)

# 1
    math_end_date = end_date + timedelta(days=1)
    contract_term_length_month = (math_end_date.year - start_date.year) * 12 + (
        math_end_date.month - start_date.month
    )
    contract_term_length_year = contract_term_length_month // 12
    contract_term_length_quarter = contract_term_length_month // 3

    contract_term_length_day = (math_end_date - start_date).days
    contract_term_nl_day = contract_term_length_day - count_leap_days(
        start_date, end_date
    )


`math_end_date`: A start date of January 1st and an end date of January 2nd will output a delta of 1 day, but the contract is really active for 2 days, which is why the `math_end_date` variable is created.

`contract_term_length_month`: To determine the number of months, code was stolen from the `dateutil` [package](https://github.com/dateutil/dateutil/blob/master/src/dateutil/relativedelta.py#L146).

`contract_term_length_year`: From `contract_term_length_month`, derive the other two variables. There are always 12 months in a year and 3 months in a quarter. The floor division operator is used by 12. and the same respectively for 3 months in a quarter.

`contract_term_length_quarter`: See above.

`contract_term_length_day`: A simple delta of the two dates will output that number, then add 1 for the final day.

`contract_term_nl_day`: Another variable is setup incase `generalize_leap_year` is set to `True`, so it can deduct leap years.

<div class="alert alert-block alert-info">
<b>Take note of the usage of floor division.</b> This is indicating that if a contract length is 1.89 years, and the <i>interval_str</i> arg was set to <i>Year</i>, then only 1 year would be considered.
</div>

In [13]:
next(get_contract_term_code)

# 2
    contract_term_mapping = {
        "Year": contract_term_length_year,
        "Quarter": contract_term_length_quarter,
        "Month": contract_term_length_month,
        "Day": (
            contract_term_nl_day if generalize_leap_year else contract_term_length_day
        ),
    }

    return contract_term_mapping[interval_str]

This dictionary brings it altogether and returns an `int` to be used in the annualize function.

Okay, detour over, back to the `annualize` function.

In [14]:
next(annualize_code)

# 4
    if print_details:
        print(contract)
        print(f"Period: {period}")
        print(f"Generalize Leap Year: {generalize_leap_year}")
        print(f"Time Interval: {interval_str} - {time_interval}")
        print(f"Contract Term: {contract_term}")
        print(f"{contract.amount} * ({time_interval}/{contract_term})")

4. If `print_details` is `True`, the details are printed out to review.

In [15]:
next(annualize_code)

# 5
    try:
        fraction = time_interval / contract_term
    except ZeroDivisionError:
        fraction = 0

    return contract.amount * (fraction)

5. Put the pieces together.

##### Example

In [16]:
annualize_example = annualize(
    contract=ContractHeader(25_000, date(2022, 1, 1), date(2023, 12, 31)),
    period=date(2023, 1, 31),
    interval_str="Day",
    generalize_leap_year=True,
    print_details=True,
)
print("*" * 55)
print(f"The ACV for `annualize_example` variable is {annualize_example}")

ContractHeader(amount=25000, start_date=datetime.date(2022, 1, 1), end_date=datetime.date(2023, 12, 31), booking_date=datetime.date(2022, 1, 1))
Period: 2023-01-31
Generalize Leap Year: True
Time Interval: Day - 365
Contract Term: 730
25000 * (365/730)
*******************************************************
The ACV for `annualize_example` variable is 12500.0


In the next section, we'll review many of the implication that can occur depending on how you build.

[Next Section](2-what-is-arr-part-2.ipynb)